In [1]:
%load_ext autoreload
%autoreload 2
import os
import torch
import json
import yaml
import tqdm
from dataset.ConcatBaselineDataset import ConcatBaselineDataset
from baseline.models.ConcatBaselineNet import ConcatBaselineNet
from dataset.MurelNetDataset import MurelNetDataset
from murel.models.MurelNet import MurelNet
from baseline.scripts.train_baseline import get_hidden_layer_list
from torch.utils.data import DataLoader
import transforms.transforms as trfm

In [2]:
DATA_DIR = "/auto/homes/bat34/VQA/"
ROOT_DIR = "/auto/homes/bat34/VQA_PartII/"
MODEL_NAME = 'murel'
CONFIG_PATH = os.path.join(ROOT_DIR, MODEL_NAME, 'scripts', MODEL_NAME + '.yaml')
with open(CONFIG_PATH) as f:
        config = yaml.load(f)
        
config['fusion']['obj_features_question']['input_dims'] = [2048, 4800]
config['fusion']['final_fusion']['input_dims'] = [2048, 4800]
config['batch_size'] = 32
        
train_dataset = MurelNetDataset(split='train', \
                        txt_enc=config['txt_enc'], \
                        bottom_up_features_dir=config['bottom_up_features_dir'], \
                        skipthoughts_dir=config['skipthoughts_dir'], \
                        processed_dir=config['processed_dir'], \
                        ROOT_DIR=ROOT_DIR, \
                        vqa_dir=config['vqa_dir'])

test_dataset = MurelNetDataset(split='test', \
                        txt_enc=config['txt_enc'], \
                        bottom_up_features_dir=config['bottom_up_features_dir'], \
                        skipthoughts_dir=config['skipthoughts_dir'], \
                        processed_dir=config['processed_dir'], \
                        ROOT_DIR=ROOT_DIR, \
                        vqa_dir=config['vqa_dir'])


test_loader = DataLoader(test_dataset, shuffle=False, \
                         num_workers=config['num_workers'], \
                         batch_size=config['batch_size'], \
                         collate_fn=test_dataset.collate_fn, \
)
    
word_vocabulary = [word for word, _ in train_dataset.word_to_wid.items()]
#model = MurelNet(config, word_vocabulary)
model = MurelNet(config, word_vocabulary)

BEST_MODEL_NAME = 'murel_txt_enc_BayesianUniSkip_batch_size_256_lr_5e-05_lr_decay_rate_0.25_unroll_steps_3_fusion_type_block'
BEST_MODEL_PATH = os.path.join(ROOT_DIR, MODEL_NAME, 'trained_models', 'best_models', \
                             BEST_MODEL_NAME, 'best_model.pth')

#Path to model weights with best performance on the validation dataset
model.load_state_dict(torch.load(BEST_MODEL_PATH)['model'])
model = model.cuda()
model = model.eval()

/home/bat34/venv/lib/python3.5/site-packages/ipykernel_launcher.py:6: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [3]:
results = []
with torch.no_grad():
    for data in tqdm.tqdm(test_loader):
        item = {\
            'question_ids': data['question_ids'].cuda(), \
            'object_features_list': data['object_features_list'].cuda(), \
            'bounding_boxes': data['bounding_boxes'].cuda(), \
            'question_lengths': data['question_lengths'].cuda(), \
        }
        inputs = item
        qids = data['question_unique_id']
        outputs = model(inputs)
        values, ans_indices = torch.max(outputs, dim=1)
        ans_indices = list(ans_indices)
        ans_indices = [tsr.item() for tsr in ans_indices]
        for qid, ans_idx in zip(qids, ans_indices):
            results.append({
                'question_id': int(qid),
                'answer': test_dataset.aid_to_ans[ans_idx]
            })

  0%|          | 0/13994 [00:00<?, ?it/s]/home/bat34/venv/lib/python3.5/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/bat34/venv/lib/python3.5/site-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
100%|██████████| 13994/13994 [44:09<00:00,  5.28it/s] 


In [7]:
RESULTS_FILE_PATH = os.path.join(ROOT_DIR, MODEL_NAME, '{}_test-2015_results.json'.format(BEST_MODEL_NAME))
with open(RESULTS_FILE_PATH, 'w') as f:
    json.dump(results, f)

In [4]:
RESULTS_FILE_PATH = os.path.join(ROOT_DIR, MODEL_NAME, '{}_test-2015_results.json'.format(BEST_MODEL_NAME))

In [ ]:
RESULTS